In [35]:
'''
Smaller Image on Top.
'''

import redis; import numpy as np; import cv2; import json; from matplotlib import pyplot as plt
def multi_window(images):
    x, y = 0, 0 
    for i, (name, image) in enumerate(images):
        cv2.namedWindow(name)
        cv2.imshow(name, image)
        cv2.moveWindow(name, x, y)
        odd = (i % 2)
        x, y = (x + odd*400), (y + abs(1-odd)*300)
    k = cv2.waitKey(0)
    cv2.destroyAllWindows()

graffiti = img1 = cv2.imread('/Users/averagehat/Dropbox/video-synth/rpi/opencv/samples/data/graf3.png')
img2 = cv2.imread('/Users/averagehat/Dropbox/video-synth/rpi/opencv/samples/data/ml.png')
# img2 = cv2.resize(_img2, img1.shape[:2][::-1])
print(f'shapes: {img1.shape} {img2.shape}') # create circle mask, center, radius, fill color, size of the border # cv2.circle(mask,(r,r), r, (255,255,255),-1)
# I want to put logo on top-left corner, So I create a ROI
rows,cols,channels = img2.shape
roi = img1[0:rows, 0:cols ]
# Now create a mask of logo and create its inverse mask also
img2gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(img2gray, 240, 255, cv2.THRESH_BINARY)
# Now black-out the area of logo in ROI
img1_bg = cv2.bitwise_and(roi,roi,mask = mask)

# Take only region of logo from logo image.
img2_fg = cv2.bitwise_and(img2,img2,mask = ~mask)

# Put logo in ROI and modify the main image
dst = cv2.add(img1_bg,img2_fg)
img1[0:rows, 0:cols ] = dst

#cv2.imshow('img1', img1)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

shapes: (640, 800, 3) (380, 308, 3)


In [36]:
multi_window([('img1_bg', img1_bg), ('img2_fg', img2_fg), ('mask', mask), ('dst', dst), ('img1', img1)])
'''
img2_fg == yellowed
img1_bg should be the inverse mask; no, it will be the original image with the cropped in black
img1_bg = cv2.bitwise_and(roi,roi,mask = mask)
so the same thing is working. just need to move the ROI I guess.
'''

'\nimg2_fg == yellowed\nimg1_bg should be the inverse mask; no, it will be the original image with the cropped in black\n\nimg1_bg = cv2.bitwise_and(roi,roi,mask = mask)\n'

In [38]:
'''
Isolate Blue,
Convert Isolated Blue -> Yellow,
Put Yellow Back on Original
'''

logo = cv2.imread('/Users/averagehat/Dropbox/video-synth/rpi/opencv/samples/data/opencv-logo-white.png')

# define range of blue color in HSV
lower_blue = np.array([110,50,50])
upper_blue = np.array([130,255,255])

# Threshold the HSV image to get only blue colors
hsv = cv2.cvtColor(logo, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv, lower_blue, upper_blue)

# Bitwise-AND mask and original image
just_blue = cv2.bitwise_and(logo,logo, mask= mask)

# res = (logo | [ 60, 255, 255])
coloured = logo.copy()
coloured[mask == 255] = (0, 255, 255)
# res = cv2.bitwise_and(logo,np.array([0, 0, 255]), mask= mask)

# img1_bg = cv2.bitwise_and(roi,roi,mask = mask)

multi_window( [('logo', logo), ('yellowed', coloured), ('mask', mask), ('InvMask', ~mask), ('just_blue', just_blue)])


In [44]:
cv2.warpAffine?

In [46]:
og_bg.shape

(222, 180, 3)

In [49]:
rows, cols, clrchannels = og_bg.shape
movementM = np.float32([[1,0,-100],[0,1,-50]])
# og_bg = cv2.bitwise_and(logo,logo, mask= ~mask)
# move the isolated color first before making the mask
moved_mask = cv2.warpAffine(mask,movementM,dsize=(cols, rows)) 
moved_bg = cv2.bitwise_and(logo,logo, mask= ~moved_mask)
moved_bg = 
multi_window( [('bg', bg), ('moved_bg', moved_bg)])

In [ ]:
'''
Move a circle using warp affine.
Working! Assumes both images want relative same place
Now is this necessary? Because we get the coordinates from arduino instead; 
we just want to use similar logic by changing how the mask is created I think
'''


graffiti = cv2.imread('/Users/averagehat/Dropbox/video-synth/rpi/opencv/samples/data/graf3.png')
x, y, r = 30, 300, 40
# crop image as a square
cropped = img2[y:y+r*2, x:x+r*2]
rows,cols,channels = cropped.shape
# roi = graffiti[0:rows, 0:cols ]
roi = graffiti[y:y+r*2, x:x+r*2]
# create a mask
mask = np.full((cropped.shape[0], cropped.shape[1]), 0, dtype=np.uint8) 
# create circle mask, center, radius, fill color, size of the border
cv2.circle(mask,(r,r), r, (255,255,255),-1)
# get only the inside pixels
fg = cv2.bitwise_or(cropped, cropped, mask=mask)
# Now create a mask of logo and create its inverse mask also
graffiti_bg = cv2.bitwise_and(roi,roi,mask =~mask)
# graffiti_bg = cv2.bitwise_or(graffiti, graffiti, mask=mask)
# background = np.full(img.shape, 255, dtype=np.uint8)
# bk = cv2.bitwise_or(background, background, mask=mask)
# Put logo in ROI and modify the main image
dst = cv2.add(graffiti_bg,fg)
new_img = graffiti.copy()
new_img[y:y+r*2, x:x+r*2] = dst
# new_img[0:rows, 0:cols ] = dst
# new_img[0:rows, 0:cols ] = dst
# ('graffiti_bg', graffiti_bg),
multi_window( [  ('new_img', new_img), ('fg', fg)])
# rows, cols, clrchannels = logo.shape
# moved_og = cv2.warpAffine(logo,movementM,dsize=(cols, rows)) 
# # moved_hsv = cv2.cvtColor(moved_og, cv2.COLOR_BGR2HSV)   # this line changes, and define hsv by creating a circle and mask
# moved_mask = cv2.inRange(moved_hsv, lower_blue, upper_blue)
# moved_bg = cv2.bitwise_and(logo,logo, mask= ~moved_mask)
# 
# # Bitwise-AND mask and original image
# just_blue = cv2.bitwise_and(moved_og,moved_og, mask= moved_mask)
# 
# # res = (logo | [ 60, 255, 255])
# coloured = moved_bg.copy()
# coloured[moved_mask == 255] = (0, 255, 255)
# # res = cv2.bitwise_and(logo,np.array([0, 0, 255]), mask= mask)
# moved_blue_overlap = moved_bg | just_blue
# 
# multi_window( [('logo', logo), ('yellowed', coloured), ('moved_mask', moved_mask),
#                ('moved_bg', moved_bg), ('just_blue', just_blue), ('moved_blue_overlap', moved_blue_overlap)])

In [55]:
'''
Finished, move all of a color etc. 
'''


# Threshold the HSV image to get only blue colors
rows, cols, clrchannels = logo.shape
moved_og = cv2.warpAffine(logo,movementM,dsize=(cols, rows)) 
moved_hsv = cv2.cvtColor(moved_og, cv2.COLOR_BGR2HSV)   # this line changes, and define hsv by creating a circle and mask
moved_mask = cv2.inRange(moved_hsv, lower_blue, upper_blue)
moved_bg = cv2.bitwise_and(logo,logo, mask= ~moved_mask)

# Bitwise-AND mask and original image
just_blue = cv2.bitwise_and(moved_og,moved_og, mask= moved_mask)

# res = (logo | [ 60, 255, 255])
coloured = moved_bg.copy()
coloured[moved_mask == 255] = (0, 255, 255)
# res = cv2.bitwise_and(logo,np.array([0, 0, 255]), mask= mask)
moved_blue_overlap = moved_bg | just_blue

multi_window( [('logo', logo), ('yellowed', coloured), ('moved_mask', moved_mask),
               ('moved_bg', moved_bg), ('just_blue', just_blue), ('moved_blue_overlap', moved_blue_overlap)])

In [ ]:

mask = cv2.inRange(hsv, lower_blue, upper_blue)
movmentM = np.float32([[1,0,100],[0,1,50]])
dst = cv.warpAffine(coloured,movementM,coloured.shape)


In [ ]:

rows,cols,channels = img2.shape
roi = img1[0:rows, 0:cols ]
# Now create a mask of logo and create its inverse mask also
img2gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(img2gray, 240, 255, cv2.THRESH_BINARY)
# Now black-out the area of logo in ROI
img1_bg = cv2.bitwise_and(roi,roi,mask = mask)

# Take only region of logo from logo image.
img2_fg = cv2.bitwise_and(img2,img2,mask = ~mask)

# Put logo in ROI and modify the main image
dst = cv2.add(img1_bg,img2_fg)
img1[0:rows, 0:cols ] = dst


In [ ]:
'''
To gray
logo2gray        = cv2.cvtColor(logo, cv2.COLOR_BGR2GRAY)
Resize
resized_image = cv2.resize(img2, img1.shape[:2]) 

# gives bad color
# plt.subplot(122),plt.imshow(final),plt.title('Final')
'''
# >>> green = np.uint8([[[0,255,0 ]]])
# >>> hsv_green = cv2.cvtColor(green,cv2.COLOR_BGR2HSV)
# >>> print hsv_green
# [[[ 60 255 255]]]
# Now you take [H-10, 100,100] and [H+10, 255, 255]
# Convert BGR to HSV